In [1]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

data_DIR = "../data/all_in_one_metrics/"
fig_DIR = "../figs/player_analysis/"
roster_DIR = "../data/rosters/" 

In [2]:
df_comb = pd.read_parquet(roster_DIR+ "NBA_Draft_Combine.parquet")

In [5]:
df_comb.columns = map(str.lower, df_comb.columns)
df_comb["season"] = df_comb["season"].astype(int)

In [7]:
df_comb.to_csv(roster_DIR+ "NBA_Draft_History.csv")

In [ ]:
def load_player_data():
    # load draft data
    df_draft = pd.read_parquet(roster_DIR+ "NBA_Draft_History.parquet")
    df_draft.columns = map(str.lower, df_draft.columns)
    df_draft["season"] = df_draft["season"].astype(int)
    df_draft = df_draft.rename(columns={"person_id":"pID"})
    df_draft = df_draft.query("season > 1995")
    print(len(df_draft))
    # load draft combine data
    df_comb = pd.read_parquet(roster_DIR+ "NBA_Draft_Combine.parquet")
    df_comb.columns = map(str.lower, df_comb.columns)
    df_comb["season"] = df_comb["season"].astype(int)
    df_comb = df_comb.rename(columns={"player_id":"pID"})
    # print(len(df_comb))
    # merge draft and combine datga
    dfm_draft = pd.merge(df_draft,df_comb,on="pID",suffixes=("_draft","_comb"))
    # print(len(dfm_draft))
    # load DARKO
    df_d = pd.read_parquet(data_DIR + "NBA_DARKO_FULL.parquet")
    df_d["season"] = df_d["season"].astype(int)
    df_d = df_d.rename(columns={"nba_id":"pID"})
    # mege DARKO and draft
    dfm = pd.merge(df_d,df_draft,on="pID",suffixes=("_dpm","_draft"))
    dfpl = dfm.query("career_game_num==1").reset_index(drop=True)
    print(len(dfpl))
    dfpl["group"] = 0
    dfpl.loc[dfpl.query("round_number==1 & round_pick<=5").index,"group"] = 1
    dfpl.loc[dfpl.query("round_number==1 & round_pick>5 & round_pick<=14").index,"group"] = 2
    dfpl.loc[dfpl.query("round_number==1 & round_pick>14 & round_pick<=21").index,"group"] = 3
    dfpl.loc[dfpl.query("round_number==1 & round_pick>21").index,"group"] = 4
    dfpl.loc[dfpl.query("round_number==2 & round_pick<=9").index,"group"] = 5
    dfpl.loc[dfpl.query("round_number==2 & round_pick>9").index,"group"] = 6
    dfpl["group"] = dfpl["group"].astype("category")
    dfpl["age"] = dfpl["age"].round()
    return df_draft,df_comb,df_d,dfm,dfpl

In [ ]:
df_draft,df_comb,df_d,dfm,dfpl = load_player_data()

In [ ]:
def get_group_data(dfplg,dfm):
    # dfplg = dfpl
    dfpl1 = dfplg.query("age <= 19")
    dfpl1["draft_age"] = 19
    pl1 = dfpl1["pID"].to_list()
    dfpl2 = dfplg.query("age == 20")
    dfpl2["draft_age"] = 20
    pl2 = dfpl2["pID"].to_list()
    dfpl3 = dfplg.query("age == 21")
    dfpl3["draft_age"] = 21
    pl3 = dfpl3["pID"].to_list()
    dfpl4 = dfplg.query("age == 22")
    dfpl4["draft_age"] = 22
    pl4 = dfpl4["pID"].to_list()
    dfpl5 = dfplg.query("age == 23")
    dfpl5["draft_age"] = 23
    pl5 = dfpl5["pID"].to_list()
    
    df1 = dfm[dfm["pID"].isin(pl1)].reset_index(drop=True)
    df1["draft_age"] = 19
    df1g = df1.query("dpm >=0").groupby("pID")
    df1c = df1g.nth[0]
    df1gm = df1.sort_values(by=["pID","dpm"]).reset_index(drop=True).groupby("pID")
    df1m = df1gm.nth[-1]
    df2 = dfm[dfm["pID"].isin(pl2)].reset_index(drop=True)
    df2["draft_age"] = 20
    df2g = df2.query("dpm >=0").groupby("pID")
    df2c = df2g.nth[0]
    df2gm = df2.sort_values(by=["pID","dpm"]).reset_index(drop=True).groupby("pID")
    df2m = df2gm.nth[-1]
    df3 = dfm[dfm["pID"].isin(pl3)].reset_index(drop=True)
    df3["draft_age"] = 21
    df3g = df3.query("dpm >=0").groupby("pID")
    df3c = df3g.nth[0]
    df3gm = df3.sort_values(by=["pID","dpm"]).reset_index(drop=True).groupby("pID")
    df3m = df3gm.nth[-1]
    df4 = dfm[dfm["pID"].isin(pl4)].reset_index(drop=True)
    df4["draft_age"] = 22
    df4g = df4.query("dpm >=0").groupby("pID")
    df4c = df4g.nth[0]
    df4gm = df4.sort_values(by=["pID","dpm"]).reset_index(drop=True).groupby("pID")
    df4m = df4gm.nth[-1]
    df5 = dfm[dfm["pID"].isin(pl5)].reset_index(drop=True)
    df5["draft_age"] = 23
    df5g = df5.query("dpm >=0").groupby("pID")
    df5c = df5g.nth[0]
    df5gm = df5.sort_values(by=["pID","dpm"]).reset_index(drop=True).groupby("pID")
    df5m = df5gm.nth[-1]

    df_prob = pd.DataFrame()
    df_prob["draft_age"] = np.arange(19,24)
    df_prob["players"] = [len(pl1),len(pl2),len(pl3),len(pl4),len(pl5)]
    df_prob["pos_players"] = [len(df1c),len(df2c),len(df3c),len(df4c),len(df5c)]

    df_prob["p_fill"] = "Total Players"
    df_prob["pp_fill"] = "Positive Players"
    df_prob["frac"] = df_prob["pos_players"]/df_prob["players"]*100

    dfc = pd.concat([df1c,df2c,df3c,df4c,df5c])
    dfc["draft_age"] = dfc["draft_age"].astype("category")
    dfmax = pd.concat([df1m,df2m,df3m,df4m,df5m])
    dfmax["draft_age"] = dfmax["draft_age"].astype("category")

    return df_prob,dfc,dfmax

In [ ]:
groups = [
    "All Drafted",
    "Picks 1-5",
    "Picks 6-14",
    "Picks 15-21",
    "Picks 22-30",
    "R2: Picks 1-9",
    "R2: Picks 10-30",
]

In [ ]:
dfp = []
dfc = []
dfmax = []
dfpa,dfca,dfmaxa = get_group_data(dfpl,dfm)
dfpa["group"] = groups[0]
dfca["group"] = groups[0]
dfmaxa["group"] = groups[0]
dfp.append(dfpa)
dfc.append(dfca)
for gi in range(1,7):
    dfplg = dfpl.query(f"group == {gi}")
    dfp1,dfc1,dfmax1 = get_group_data(dfplg,dfm)
    dfp1["group"] = groups[gi]
    dfc1["group"] = groups[gi]
    dfmax1["group"] = groups[gi]
    dfp.append(dfp1)
    dfc.append(dfc1)
    dfmax.append(dfmax1)
dfp = pd.concat(dfp)
dfc = pd.concat(dfc)
dfmax = pd.concat(dfmax)
dfp["group"] = pd.Categorical(dfp['group'], categories=groups)

In [ ]:
dfmax[["player_name_dpm","dpm"]]

In [ ]:
p = (
    ggplot(dfp.query("group == 'All Drafted'"))
    + aes(x="draft_age")
    + geom_bar(aes(y="players", fill="p_fill"),stat="identity", alpha=0.5)
    + geom_bar(aes(y="pos_players", fill="pp_fill"),stat="identity", alpha=0.6)
    + geom_text(aes(label = "frac", y="pos_players"),size=8,nudge_y=+2,format_string ="{:0.1f}%")
    + scale_fill_manual(values = ["grey","black"])
    + theme_idv
    + theme(
        plot_caption=element_text(ha="left"),
        legend_title=element_blank(),
        plot_subtitle=element_text(size=12),
        # legend_position = [0.22,0.85],
    )
    + labs(
        x="Draft Age",
        y="# Players",
        title=f"Positive DARKO Players",
        subtitle=f"From 1996-97 NBA Season\nPlayer has to have atleast 1 game of DPM >=0",
        caption="@SravanNBA | source:@kmedved (darko.app)",
    )
)
p.save(fig_DIR + f"draft_frac_pos_dpm.png", dpi=300, verbose=False)
p

In [ ]:
colors = [
    '#000000',
    '#0057e7',
    '#d62d20',
    '#008744',
    '#ffa700',
    '#9AA0A6',
    '#4d1b7b',
]

In [ ]:
dfc["overall_pick"]

In [ ]:
p = (
    ggplot(dfc.query("group == 'All Drafted'"),aes(x="draft_age",y="career_game_num", fill="draft_age"))
    + geom_boxplot(outlier_shape="", notch=True, varwidth=True, notchwidth=0.6)
    # + geom_violin(draw_quantiles=[0.25,0.5,0.75,0.9], alpha=0.25, show_legend=False, size=1)
    # + geom_jitter(width=0.1, height=0.2)
    # + geom_dotplot(binaxis = "y", stackdir = "center", dotsize=0.1, stackgroups=True)#, position = "dodge"
    + scale_fill_manual(values = colors[1:], name="Draft Age")
    + theme_idv
    + theme(
        # legend_title=element_blank()
    )
    # + ylim(0,400)
    # + ylim(0,240)
    + coord_cartesian(ylim=[0,300])
    + labs(
        x="Draft Age",
        y="Career Game #",
        title="Career Game # of First Positive DPM Game",
        subtitle="From 1996-97 NBA Season Onwards",
        caption="@Sravan | source:kmedved (darko.app)",
    )
)
p.save(fig_DIR + "draft_first_pos_dpm_box.png", dpi=300, verbose=False)
p

In [ ]:
p = (
    ggplot(dfc.query("group == 'All Drafted'"),aes(x="draft_age",y="career_game_num", fill="draft_age"))
    # + geom_boxplot(outlier_shape="", notch=True, varwidth=True, notchwidth=0.6)
    + geom_violin(draw_quantiles=[0.25,0.5,0.75], alpha=0.5, show_legend=False, size=1)
    # + geom_jitter(width=0.1, height=0.2)
    # + geom_dotplot(binaxis = "y", stackdir = "center", dotsize=0.1, stackgroups=True)#, position = "dodge"
    + scale_fill_manual(values = colors[1:], name="Draft Age")
    + theme_idv
    + theme(
        # legend_title=element_blank()
    )
    # + ylim(0,400)
    # + ylim(0,240)
    + coord_cartesian(ylim=[0,300])
    + labs(
        x="Draft Age",
        y="Career Game #",
        title="Career Game # of First Positive DPM Game",
        subtitle="From 1996-97 NBA Season Onwards",
        caption="@Sravan | source:kmedved (darko.app)",
    )
)
p.save(fig_DIR + "draft_first_pos_dpm_violin.png", dpi=300, verbose=False)
p

In [ ]:
dfc1 = dfc.query("group == 'All Drafted'")
dfa = []
for da in range(19,24):
    des1 = dfc1.query(f"draft_age == {da}").describe().T
    dfa.append(des1.loc["career_game_num"])
df_des = pd.DataFrame(dfa)
df_des = df_des.set_index(np.arange(19,24)).reset_index()
df_des = df_des.rename(columns={"index":"Draft Age"})
df_des = df_des.round(0).astype(int)
df_des.index +=1
print(df_des.to_markdown())

In [ ]:
dfp.query("group == 'R2: Picks 10-30'")

In [ ]:
p = (
    ggplot(dfp)
    + aes(x="draft_age", y = "frac", group ="group", color = "group")
    + geom_point()
    + geom_line(size=1.5)
    + scale_color_manual(values = colors)
    + scale_y_continuous(breaks=range(30,100,10))
    # + scale_color_cmap(cmap_name="tab10")
    # + scale_color_brewer(type="qual", palette=2)
    + theme_idv
    + theme(
        plot_caption=element_text(ha="left"),
        legend_title=element_blank(),
        plot_subtitle=element_text(size=12),
        # legend_position = [0.22,0.85],
    )
    + labs(
        x="Draft Age",
        y="% of Players",
        title=f"% of Positive DARKO Players",
        subtitle=f"From 1996-97 NBA Season | Player has to have atleast 1 game of DPM >=0",
        caption="@SravanNBA | source:@kmedved (darko.app)",
    )
)
p.save(fig_DIR + f"draft_per_pos_dpm.png", dpi=300, verbose=False)
p

In [ ]:
p = (
    ggplot(dfmax)
    + aes(x="draft_age",y="dpm", fill="draft_age")
    + geom_boxplot(outlier_shape="", notch=True, varwidth=True, notchwidth=0.6)
    # + geom_violin(draw_quantiles=[0.25,0.5,0.75,0.9], alpha=0.25, show_legend=False, size=1)
    # + geom_jitter(width=0.1, height=0.2)
    # + geom_dotplot(binaxis = "y", stackdir = "center", dotsize=0.1, stackgroups=True)#, position = "dodge"
    + scale_fill_manual(values = colors[1:], name="Draft Age")
    + theme_idv
    + theme(
        # legend_title=element_blank()
    )
    # + ylim(0,400)
    # + ylim(0,240)
    + coord_cartesian(ylim=[-2,3])
    + labs(
        x="Draft Age",
        y="DPM",
        title="Maximum DPM of Player's Career",
        subtitle="From 1996-97 NBA Season Onwards",
        caption="@Sravan | source:kmedved (darko.app)",
    )
)
p.save(fig_DIR + "draft_max_dpm_box.png", dpi=300, verbose=False)
p

In [ ]:
dfa = []
for da in range(19,24):
    des1 = dfmax.query(f"draft_age == {da}").describe().T
    dfa.append(des1.loc["dpm"])
df_des = pd.DataFrame(dfa)
df_des = df_des.set_index(np.arange(19,24)).reset_index()
df_des = df_des.rename(columns={"index":"Draft Age"})
df_des = df_des.round(1)
df_des.index +=1
print(df_des.to_markdown())

In [ ]:
dfb = []
df_des1 = df_des.copy()
df_des1["group"] = groups[0]
df_des1 = df_des1.rename(columns={"Draft Age":"draft_age"})
dfb.append(df_des1)
for gi in groups[1:]:
    dfa = []
    dfmax1 = dfmax.query(f"group == '{gi}'")  
    for da in range(19,24):
        des1 = dfmax1.query(f"draft_age == {da}").describe().T
        dfa.append(des1.loc["dpm"])
    df_des = pd.DataFrame(dfa)
    df_des = df_des.set_index(np.arange(19,24)).reset_index()
    df_des = df_des.rename(columns={"index":"draft_age"})
    df_des = df_des.round(1)
    df_des["group"] = gi
    dfb.append(df_des)
# df_des.index +=1
# print(df_des.to_markdown())
dfmax_des = pd.concat(dfb)

In [ ]:
p = (
    ggplot(dfmax_des)
    + aes(x="draft_age", y = "50%", group ="group", color = "group")
    + geom_point()
    + geom_line(size=1.5)
    + scale_color_manual(values = colors)
    # + scale_y_continuous(breaks=range(30,100,10))
    # + scale_color_cmap(cmap_name="tab10")
    # + scale_color_brewer(type="qual", palette=2)
    + theme_idv
    + theme(
        plot_caption=element_text(ha="left"),
        legend_title=element_blank(),
        plot_subtitle=element_text(size=12),
        # legend_position = [0.22,0.85],
    )
    + labs(
        x="Draft Age",
        y="DPM",
        title=f"Median (50% Percentile) of Max DPM",
        subtitle=f"From 1996-97 NBA Season",
        caption="@SravanNBA | source:@kmedved (darko.app)",
    )
)
p.save(fig_DIR + f"draft_mode_max_dpm.png", dpi=300, verbose=False)
p

In [ ]:
# df = df_d[df_d["nba_id"].isin(player_list)].reset_index(drop=True)
# df

In [ ]:
# dfg = df.groupby(["player_name","nba_id"])
# keys = list(dfg.groups)
# key = keys[0]
# df1 = dfg.get_group(key)
# dfg.nth[0]